In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')

In [2]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [4]:
# parameters
customer_id = '5'
formatted_attribute = 'Length'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Window Blinds"""
buckets = """Package Quantity"""

regex_pattern=r'(?i)(x\s?\d+.?\d*\".?D)|(.?.?.?.?.?Depth.?.?.?.?)()'

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
curated_by=762

In [5]:
regex_pattern_inch = r"(?i)(\d+.?Inch)|()"
regex_pattern_wreath = r"(?i)(Wreath)|()"
regex_pattern = r"(?i)(\d+\.?\d*\s?\\\"\s?diam)|(\d+\.\d*\s?\\\"\s?diam)|(\s\.\d*\s?\\\"\s?diam)|(\d+.?\d*\/\d*\s?\\\"\s?diam)|()"
regex_pattern_general = r"(?i)(\d+\\\")|()"
regex_pattern_height = r"(?i)(\\\" H)|()"

In [14]:
# df = query_from_file(file_name='Curate_wipe/BusinessIntelligence/Platform Queries/Bucket Attribute Values for All active Strategies.sql', params=params)
df = query_from_file(file_name='query/Delete_Motion_Max_Min_temp_date.sql', params=params)
# df=df[df['value']=='n/a']
print(len(df))
df.head(1)

222378


attributes snake_case_name external_id  \
0     Length          length    10081483   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             long_desc  \
0  Get organized with the Mega Expand-A-Shelf Tiered Organizer that allows you to see everything on your shelves. Perfect for a pantry, kitchen cabinet or even a utility room, the shelf expands from 13\" to 25\" to offer complete cabinet storage. Keep things neat and organized so you can find what you need easily with the Mega Expand-A-Shelf Tiered Organizer The step design uses wasted vertical space and requires no tools for a custom fit Expandable length fits any shelf between 13\" and 25\" Plastic Wipe clean Measures 13\" W x 12\" D x 6\" H (without expanding); 25\" W x 12\" D x 6\" H (expanded) Imported   

  curated_date  customers_id  \
0   2021-02-08             5   

                                                    buckets value resolution  
0  Storage & Organization Products Components & Accessories   n/a   standard

In [13]:
df['value'].explode().value_counts()[-1000:-500]

44.9 in                                                       1
["36 in","65.13 in"]                                          1
["19.69 in","22.44 in","33.46 in","40.16 in"]                 1
["18 in","33 in"]                                             1
["11 in","7 in"]                                              1
["24 in","6.89 in"]                                           1
["12 in","15 in"]                                             1
["75.38 in","80.25 in"]                                       1
21.34 in                                                      1
["4.9 in","5.3 in"]                                           1
["25 in","49 in"]                                             1
["20 in","24 in","30 in","48 in"]                             1
16.79 in                                                      1
["26 in","36 in","80 in","84 in","96 in"]                     1
["73 in","88 in"]                                             1
["10.25 in","30 in"]                    

In [33]:
df['general_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_general, x))
general=df[df['general_matches'].astype(str)!='[]']
len(general)

53940

In [37]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
diam=df[df['matches'].astype(str)!='[]']
len(df[df['matches'].astype(str)!='[]'])

241

In [57]:
diam=diam[diam['buckets'].astype(str)!='Lamps']
len(diam)

173

In [58]:
matches=diam

In [59]:
matches[curation_col]=matches['matches']

In [11]:
df['inch_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_inch, x))
inch=df[df['inch_matches'].astype(str)!='[]']
len(inch)

1270

In [15]:
df['height_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
height=df[df['height_matches'].astype(str)!='[]']
len(height)

34001

In [20]:
gen=df[(df['general_matches'].astype(str)!='[]')&(df['wreath_matches'].astype(str)!='[]')&(df['height_matches'].astype(str)=='[]')]
len(gen)

170

In [23]:
na_upload=gen
na_upload[curation_col]=r'n/a'

In [34]:
gen['general_matches']=gen['general_matches'].apply(lambda measure: re.sub(r'\\\"',' in',str(measure)))
gen['general_matches']=gen['general_matches'].apply(lambda measure: re.sub(r'\\','',str(measure)))
gen['general_matches'].explode().value_counts()

['18 in']                                                51
['5 in', '25 in']                                        23
['18 in', '5 in']                                        20
['16 in']                                                13
['75 in']                                                11
['5 in', '24 in']                                        11
['24 in']                                                10
['4 in']                                                  9
['24 in', '30 in']                                        7
['5 in']                                                  6
['20 in']                                                 6
['5 in', '22 in']                                         5
['24 in', '4 in']                                         5
['5 in', '20 in']                                         5
['29 in', '43 in']                                        4
['102 in', '7 in', '54 in', '5 in', '9 in']               4
['30 in']                               

In [60]:
gen

attributes snake_case_name external_id  \
3069    Diameter        diameter    40027616   
3075    Diameter        diameter    40027784   
3078    Diameter        diameter    40027838   
3080    Diameter        diameter    40029061   
3082    Diameter        diameter    40029221   
3087    Diameter        diameter    40029474   
3088    Diameter        diameter    40029481   
3089    Diameter        diameter    40029498   
4020    Diameter        diameter    40834078   
4071    Diameter        diameter    40893631   
4072    Diameter        diameter    40893648   
4074    Diameter        diameter    40893662   
4895    Diameter        diameter    41694671   
4906    Diameter        diameter    41698617   
5218    Diameter        diameter    41964668   
5367    Diameter        diameter    42066101   
6831    Diameter        diameter    42970989   
6833    Diameter        diameter    42971016   
6834    Diameter        diameter    42971023   
7375    Diameter        diameter    43290819   
10491   Diameter        diameter    45451072   
10894   Diameter        diameter    45584282   
14359   Diameter        diameter    46701053   
15451   Diameter        diameter    47148390   
16885   Diameter        diameter    47365179   
17719   Diameter        diameter    47532649   
19048   Diameter        diameter    60295361   
19928   Diameter        diameter    60740724   
19932   Diameter        diameter    60740762   
19936   Diameter        diameter    60740823   
24379   Diameter        diameter    62113199   
24380   Diameter        diameter    62113229   
24381   Diameter        diameter    62113274   
24382   Diameter        diameter    62113281   
24383   Diameter        diameter    62113298   
24385   Diameter        diameter    62113335   
24388   Diameter        diameter    62113373   
24389   Diameter        diameter    62113380   
24392   Diameter        diameter    62113502   
24393   Diameter        diameter    62113533   
24394   Diameter        diameter    62113540   
24395   Diameter        diameter    62113571   
26753   Diameter        diameter    62609647   
26759   Diameter        diameter    62609814   
26766   Diameter        diameter    62610209   
26858   Diameter        diameter    62615174   
26861   Diameter        diameter    62615204   
28699   Diameter        diameter    62924993   
29716   Diameter        diameter    63163346   
29905   Diameter        diameter    63191509   
31065   Diameter        diameter    63411386   
34476   Diameter        diameter    64561905   
36187   Diameter        diameter    64903439   
36225   Diameter        diameter    64903811   
37019   Diameter        diameter    65158609   
38019   Diameter        diameter    65359709   
38516   Diameter        diameter    65446164   
42904   Diameter        diameter    66182672   
42948   Diameter        diameter    66185536   
43116   Diameter        diameter    66218647   
43121   Diameter        diameter    66218692   
43127   Diameter        diameter    66218753   
43140   Diameter        diameter    66218883   
43147   Diameter        diameter    66219002   
43606   Diameter        diameter    66281863   
43607   Diameter        diameter    66281870   
44458   Diameter        diameter    66430230   
44472   Diameter        diameter    66430605   
44480   Diameter        diameter    66430827   
44490   Diameter        diameter    66431107   
44671   Diameter        diameter    66450139   
44672   Diameter        diameter    66450146   
44673   Diameter        diameter    66450153   
44674   Diameter        diameter    66450160   
46145   Diameter        diameter    66648413   
46593   Diameter        diameter    66687368   
46956   Diameter        diameter    66729747   
46957   Diameter        diameter    66729754   
46958   Diameter        diameter    66729761   
47981   Diameter        diameter    66908845   
49264   Diameter        diameter    67157488   
49268   Diameter        diameter    67157525   
49875   Diameter      

In [41]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
#     data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())
#     data[['length', 'width']]=data['matches'].str.split(" x ",expand=True,)

In [42]:
def drop_characters(data):
    data['length']=data['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['width']=data['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('"', ''))
    data['width']=data['width'].apply(lambda quote: quote.replace('"', ''))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('-',''))
    data['width']=data['width'].apply(lambda quote: quote.replace('-',''))

In [43]:
def replace_none(data):
    data['width']=data['width'].apply(lambda x: 0 if x == '' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == '' else x)
    data['width']=data['width'].apply(lambda x: 0 if x == 'None' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == 'None' else x)

In [44]:
def add_quotes(data):
    data['length']=data['length'].apply(lambda x: str(x) + "'")
    data['width']=data['width'].apply(lambda x: str(x) + "'")

In [45]:
def min_max(data):
    data['length']=data['length'].astype(float)
    data['width']=data['width'].astype(float)
    data['Max'] = data[['length', 'width']].values.max(1)
    data['Min'] = data[['length', 'width']].values.min(1)

In [46]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split(' ')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [54]:
# lamps[['length', 'width']]=lamps['matches'].str.split(",",expand=True,)
lamps['matches']=lamps['matches'].astype(str)

In [55]:
lamps

attributes snake_case_name external_id  \
8367    Diameter        diameter    43979059   
8660    Diameter        diameter    44161927   
8661    Diameter        diameter    44161934   
9027    Diameter        diameter    44387570   
12344   Diameter        diameter    46306753   
12346   Diameter        diameter    46306784   
12358   Diameter        diameter    46307095   
12361   Diameter        diameter    46307798   
12406   Diameter        diameter    46309518   
13464   Diameter        diameter    46423962   
14840   Diameter        diameter    46910875   
16702   Diameter        diameter    47350458   
16704   Diameter        diameter    47350496   
16706   Diameter        diameter    47350533   
17026   Diameter        diameter    47372641   
17186   Diameter        diameter    47415607   
17197   Diameter        diameter    47416222   
17418   Diameter        diameter    47459908   
17420   Diameter        diameter    47459984   
17422   Diameter        diameter    47460171   
17429   Diameter        diameter    47461352   
17706   Diameter        diameter    47532410   
17965   Diameter        diameter    47612402   
17966   Diameter        diameter    47612419   
17967   Diameter        diameter    47612426   
17969   Diameter        diameter    47612457   
18078   Diameter        diameter    47638457   
18079   Diameter        diameter    47638464   
18083   Diameter        diameter    47638624   
18085   Diameter        diameter    47638648   
21423   Diameter        diameter    61130289   
30254   Diameter        diameter    63227185   
30319   Diameter        diameter    63227857   
30327   Diameter        diameter    63227949   
30330   Diameter        diameter    63227970   
30342   Diameter        diameter    63228090   
33068   Diameter        diameter    63900279   
33073   Diameter        diameter    63900453   
33946   Diameter        diameter    64322223   
35425   Diameter        diameter    64728872   
37447   Diameter        diameter    65273050   
39881   Diameter        diameter    65658963   
39890   Diameter        diameter    65659076   
39895   Diameter        diameter    65659120   
39896   Diameter        diameter    65659144   
39898   Diameter        diameter    65659182   
40161   Diameter        diameter    65708903   
55397   Diameter        diameter    68236021   
64003   Diameter        diameter    69433917   
66748   Diameter        diameter    69507042   
67362   Diameter        diameter    69514268   
68643   Diameter        diameter    69529617   
69927   Diameter        diameter    69549180   
70904   Diameter        diameter    69589109   
70910   Diameter        diameter    69589115   
71282   Diameter        diameter    69592296   
71533   Diameter        diameter    69593482   
71544   Diameter        diameter    69593495   
71547   Diameter        diameter    69593498   
71562   Diameter        diameter    69593518   
72688   Diameter        diameter    69604850   
72705   Diameter        diameter    69604876   
73131   Diameter        diameter    69614273   
77267   Diameter        diameter    69677588   
77268   Diameter        diameter    69677589   
77271   Diameter        diameter    69677592   
80610   Diameter        diameter    76665913   
80713   Diameter        diameter    77357892   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [ ]:
if lamps['matches'].find(',')>-1:
    lamps[['length', 'width']]=lamps['matches'].apply(lambda x: x.split(','))

In [48]:
lamps['matches']

8367                    [15\" diam, 17\" diam]
8660                [18\" diam, 17-1/2\" diam]
8661                    [16\" diam, 14\" diam]
9027                               [16\" diam]
12344                   [15\" diam, 10\" diam]
12346                              [14\" diam]
12358                   [15\" diam, 10\" diam]
12361                   [15\" diam, 10\" diam]
12406                               [7\" diam]
13464       [11\" diam, 5.5\" diam, 13\" diam]
14840                             [6.5\" diam]
16702                              [18\" diam]
16704    [18\" diam, 3.25\" diam, 12.5\" diam]
16706                  [1.5\" diam, 10\" diam]
17026                              [17\" diam]
17186                              [10\" diam]
17197                              [15\" diam]
17418                              [15\" diam]
17420                              [15\" diam]
17422                              [17\" diam]
17429                              [10\" diam]
17706        

In [ ]:
split_col(df) #might throw an error and have to drop nane

In [ ]:
df['width']=df['width'].astype(str).apply(lambda x: 0 if x == 'nan' else x)
df

In [ ]:
df['length']=df['length'].astype(str)
df['width']=df['width'].astype(str)
df['length']=df['length'].apply(lambda quote: quote.replace('-', ' '))
df['width']=df['width'].apply(lambda quote: quote.replace('-', ' '))
df['length']=df['length'].apply(lambda quote: quote.replace('inch', ''))
df['width']=df['width'].apply(lambda quote: quote.replace('inch', ''))
df

In [ ]:
df['length']=df['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\w*\.\w*\.|\-|\'",'',str(measure)))
df['width']=df['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\w*\.\w*\.|\-|\'",'',str(measure)))
df

In [ ]:
a=df['length'].to_list()
c=[]
for i in range(len(a)):
    b=float(sum(Fraction(s) for s in a[i].split()))
    c.append(b)
df['length']=c
df

In [ ]:
a=df['width'].to_list()
c=[]
for i in range(len(a)):
    b=float(sum(Fraction(s) for s in a[i].split()))
    c.append(b)
df['width']=c
df

In [ ]:
# x=df
# a=x['length'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)
# x['new'] = c
# x

In [ ]:
# a=x['length'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)

In [ ]:
# remove_fractions(df['length'])

In [ ]:
# x=df
# a=eval(x['length'][3:4].astype(str))
# # numbers = eval(a)

In [ ]:
# float(sum(Fraction(s) for s in a))

In [ ]:
# from fractions import Fraction
# float(sum(Fraction(s) for s in '  12 1/2'.split()))

In [ ]:
# x=df
# x['length'] = x['length'].apply(lambda quote: quote.replace('-', ''))
# x['length'] = x['length'].apply(lambda quote: quote.replace('inch', ''))
# # x['length'] = x['length'].apply(lambda x: x.str)
# x

In [ ]:
# x[['first', 'second']]=x['length'].str.split("/",expand=True,)
# x[['third', 'fourth']= x['first'].split()
# result = x['third'] + (float(x['first'])/float(x['second']))
# result

In [ ]:
# num,den = s.split( '/' )
# wh, num = num.split()
# result = wh + (float(num)/float(den))

In [ ]:
# a=x['length'][3:4]
# a.str.split(' ')
# b= [[" ".join(tag.split(" ")[:2]) , " ".join(tag.split(" ")[2:])] for tag in a][0]
# b

In [ ]:
# c=b[1]
# c.split('/')

In [ ]:
# b[0]
# b

In [ ]:
# a.apply(lambda x: sum(Fraction(x.split('/'))))

In [ ]:
# float(sum(Fraction(s) for s in x['length'].split()))
# x['length']=x['length'].apply(lambda a: float(sum(Fraction(a.split()))))

In [ ]:
# for i in x['length']:
#     if x['length'][i].find('/') > -1:
#         float(sum(Fraction(s) for s in x['length'][i].split()))
#     else:
#         x['length']

In [ ]:
# float(sum(Fraction(s) for s in x['length'][i].split()))
# .apply(lambda x: float(sum(Fraction((s) for s in numbers.split()))))

In [ ]:
# def fraction_combo(numbers):
#     if numbers.find('/') > -1:
#         numbers.apply(lambda x: float(sum(Fraction((s) for s in numbers.split()))))
#         #numbers = literal_eval(numbers)
#         #inte, fract = numbers.split(' ')
#         #nom, den = fract.split('/')
#         #tmp = float(nom)/float(den)
#         #num = int(inte) + tmp
#         return str(numbers)
#     else:
#         return numbers

In [ ]:
# a=x['length']

In [ ]:
# remove_fractions(df)

In [ ]:
# x['length'] = x['length'].apply(lambda x: remove_fractions(x))
# x.head

In [ ]:
# x['length'] = x['length'].apply(lambda x: x.split(' '))
# x.head()

In [ ]:
min_max(df) #error base 10 when there is only a value for max and not min

In [ ]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [ ]:
na_to_gsheet(df)

In [ ]:
# # range function builder
# string = range_string.replace(metric, '')
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

In [ ]:
df

In [ ]:
matches=df
matches['Max']=matches['Max'].astype(str).apply(lambda quote: quote.replace('.0', ''))
matches

In [ ]:
matches[curation_col] = matches['Max'].astype(str)+' in'
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

In [ ]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, matches)